# Unity Catalog: Data Monitoring

## Bloque 9: Monitor de Series Temporales - DEMO


### 9.0 Caso de uso
Analicemos un caso de uso en el sector minorista, donde una de las capas más importantes es la tabla `silver_transaction`, que combina datos de las tablas *bronze* de origen y afecta a las tablas *gold* posteriores.  
El esquema de datos utilizado en la demostración es el siguiente:

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/lhm/lhm_data.png" width="600px"/>

Los analistas de datos utilizan estas tablas para generar informes y tomar diversas decisiones empresariales.  
Recientemente, un analista intenta determinar el `PreferredPaymentMethod` más popular.  
Al consultar la tabla `silver_transaction`, descubre que existe un número considerable de transacciones con `PreferredPaymentMethod` en `null`, y comparte una captura del problema:

![image3](https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/lhm/lhm_payment_type.png?raw=true)

En este punto, es posible que te plantees varias preguntas, como por ejemplo:  
1. ¿Qué porcentaje de valores `null` se ha introducido en esta columna? ¿Es algo habitual?  
2. Si no lo es, ¿cuál fue la causa raíz de este problema de integridad?  
3. ¿Qué activos descendentes podrían haberse visto afectados por este problema?

### 9.1 Dependencias para el ejercicio

In [0]:
%pip install databricks-sdk==0.40.0
%restart_python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 629.7/629.7 kB 24.6 MB/s eta 0:00:00
  Attempting uninstall: databricks-sdk
    Found existing installation: databricks-sdk 0.49.0
    Not uninstalling databricks-sdk at /databricks/python3/lib/python3.12/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-d1201ae1-5cf7-41ed-9467-5a55f33287d5
    Can't uninstall 'databricks-sdk'. No files were found to uninstall.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
databricks-connect 17.2.2 requires databricks-sdk>=0.46.0, but you have databricks-sdk 0.40.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
%run ./src/08_DataGeneration


# 01-DataGeneration
Within this notebook, generate the following dataset:

1. The user bronze table, 
2. The product bronze table, 
3. The daily transaction table

## The Highlevel Overview of the Data Dictionary

<img src="https://raw.githubusercontent.com/databricks-demos/dbdemos-resources/main/images/product/lhm/lhm_data.png" width="600px" style="float:right"/>

## Configuration file

Please change your catalog and schema here to run the demo on a different catalog.

<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=DBSQL&org_id=1330931038747594&notebook=%2Fconfig&demo_name=lakehouse-monitoring&event=VIEW&path=%2F_dbdemos%2FDBSQL%2Flakehouse-monitoring%2Fconfig&version=1">

### License
This demo installs the following external libraries on top of DBR(ML):


| Library | License |
|---------|---------|
| faker      | [MIT](https://faker.readthedocs.io/en/master/)     |






# Technical Setup notebook. Hide this cell results
Initialize dataset to the current user and cleanup data when reset_all_data is set to true

Do not edit

...

### Genearate the user table

Schema for the User Table:
- **UserID**: Unique identifier for the user
- **Username**: User's chosen display name
- **Email**: User's email address
- **PasswordHash**: Hashed version of the user's password
- **FullName**: User's full name
- **DateOfBirth**: User's date of birth
- **Gender**: User's gender
- **PhoneNumber**: User's contact number
- **Address**: User's primary address
- **City**: User's city of residence
- **State**: User's state of residence
- **Country**: User's country of residence
- **PostalCode**: User's postal code
- **RegistrationDate**: Date when the user registered on the platform
- **LastLoginDate**: Date and time of the user's last login
- **AccountStatus**: Status of the user's account (e.g., active, suspended)
- **UserRole**: Role of the user (e.g., customer, admin)
- **PreferredPaymentMethod**: User's preferred payment method
- **TotalPurchaseAmount**: Total amount spent by the user
- **NewsletterSubscription**: Whether the user is subscribed to the newsletter (yes/no)
- **Wishlist**: List of product IDs in the user's wishlist
- **CartItems**: List of product IDs currently in the user's cart

### Genearate the product table

Schema for the Product Table:
- **ProductID**: Unique identifier for the product
- **ProductName**: Name of the product
- **Category**: Category to which the product belongs
- **SubCategory**: Subcategory of the product
- **Brand**: Brand of the product
- **Description**: Detailed description of the product
- **Price**: Price of the product
- **Discount**: Discount on the product (if any)
- **StockQuantity**: Number of items available in stock
- **SKU**: Stock Keeping Unit identifier
- **ProductImageURL**: URL of the product image
- **ProductRating**: Average rating of the product
- **NumberOfReviews**: Number of reviews for the product
- **SupplierID**: Unique identifier for the supplier
- **DateAdded**: Date when the product was added to the inventory
- **Dimensions**: Dimensions of the product (L x W x H)
- **Weight**: Weight of the product
- **Color**: Color of the product
- **Material**: Material of the product
- **WarrantyPeriod**: Warranty period of the product
- **ReturnPolicy**: Return policy for the product
- **ShippingCost**: Cost of shipping the product
- **ProductTags**: Tags associated with

### Genearate the transactions table

Schema for the transactions Table:
- **TransactionID**: Unique identifier for the transcation
- **UserID**: Unique identifier for the user
- **ProductID**: Unique identifier for the product
- **TransactionDate**: Transcation timestamp
- **Quantity**: Number of the items ordered
- **UnitPrice**: Unit price for the item ordered
- **TotalPrice**: Total amount of the purchase
- **PaymentMethod**: The payment method
- **ShippingAddress**: The shipping address for the order
- **LoyaltyPointsEarned**: The loyalty points earned per purchase (10% of total amount)
- **GiftWrap**: yes or no on gift wrap
- **SpecialInstructions**: Any other special intructions

#### Customizations:
- Date Range: Transactions are generated for each day within the specified date range.
- Seasonality: Different seasons have different base transaction volumes.
- Weekday/Weekend: Weekend transaction volumes are higher than weekdays.
- Marketing Campaigns: Specific days can have higher transaction volumes due to marketing campaigns.

## Generate Gold Tables

### 9.2 Vemos el Data Set

In [0]:
%sql 
-- To setup monitoring, load in the silver_transaction dataset
SELECT * from main_monitoring.demo_monitoring.silver_transaction limit 10;

ProductID,UserID,TransactionID,TransactionDate,Quantity,UnitPrice,TotalPrice,PaymentMethod,ShippingAddress,LoyaltyPointsEarned,GiftWrap,SpecialInstructions,Username,Email,PasswordHash,FullName,DateOfBirth,Gender,PhoneNumber,Address,City,State,Country,PostalCode,RegistrationDate,LastLoginDate,AccountStatus,UserRole,PreferredPaymentMethod,TotalPurchaseAmount,NewsletterSubscription,Wishlist,CartItems,ProductName,Category,SubCategory,Brand,Description,Price,Discount,StockQuantity,SKU,ProductImageURL,ProductRating,NumberOfReviews,SupplierID,DateAdded,Dimensions,Weight,Color,Material,WarrantyPeriod,ReturnPolicy,ShippingCost,ProductTags,TempDate,Campaign_flag
53b53445-bb17-4b3c-ad66-74282ec5acf1,d48e39e3-d9ef-4a19-a4a1-82502ea874a3,8f9dc242-7964-4da4-ac2f-30a8695bc16c,2025-10-02T16:37:57.194Z,1.0,1915.98,1915.97998046875,PayPal,"8856 Patrick Center Alisontown, KY 64161",192,no,Set respond fact ready as nation.,bryantkimberly,mccoylindsay@example.com,d84ecdcd82793490325b216f2acf11484b8db913502481114806f5738222003b,Jessica Frazier,2005-11-15,Other,309-257-8705,"8856 Patrick Center Alisontown, KY 64161",Berrymouth,Hawaii,Egypt,33494,2025-03-25,2025-08-18,Suspended,Customer,PayPal,547.81,true,"List(8059962e-881c-4501-8394-f7ad859b5df4, d0c6e50d-c9ff-48ef-93fe-7d93fd456acb)",List(5bddf4a6-a05e-4c58-815f-7541d298f737),Cookbook,Books,Non-Fiction,BrandR,Engaging story that captivates readers.,1915.98,0.0,226,YnV-33115378,https://placekitten.com/399/2,3.4000000953674316,1492,36ac73f6-1738-4369-8d78-b5b13bac9574,2025-10-07,88.41 x 58.84 x 11.04,36.58,DarkOrange,Plastic,2 months,30 days,25.9,"List(five, artist, PM)",2025-10-02,false
ec9abeae-0b13-4250-8fe5-d391519e02fe,23e810c0-7272-413f-a9f5-4654db0777fe,0897b8c3-2d87-427a-9dbb-98bfa07e3e06,2025-10-02T00:00:11.453Z,4.0,691.84,2767.360107421875,Credit Card,"096 Gonzalez Spurs Apt. 741 Lake Marc, IA 30826",277,null,,steven36,xjohnson@example.com,c0c28d7dc1110a4a33939071df6fb918d44ed09a3f2f582e6560f02e6890a310,Brandon Adams,1997-03-18,Female,001-847-339-2537x610,"096 Gonzalez Spurs Apt. 741 Lake Marc, IA 30826",Sarahton,Utah,Mexico,20290,2021-08-13,2025-05-27,Active,Customer,Bank Transfer,4124.46,true,"List(7c0e1b02-b290-4146-86d9-05d88b84f5ca, 36ff3679-9174-4def-bdd9-a85ad54b111e, f077f171-540d-4dc4-bf5f-36b8ef04d106, 96b6dc90-1ce6-4596-b13b-b10b2f5f4d63, cb74df82-4b53-42bc-a1c0-bf9ee6b705c7, 5a2a1a39-8110-422c-bf16-38d82231bbe3, 7dd318bb-da10-4262-a194-d3d355e0254c)",List(a176972f-8a12-4d87-8cd7-f127d95b8d04),Cutting Board,Home & Kitchen,Bedding,BrandGG,Essential appliance for modern homes.,691.84,0.25,597,oKS-46512038,https://picsum.photos/738/985,2.9000000953674316,524,3fa7c535-f9e0-4275-9799-c7db867c7712,2024-04-04,28.56 x 32.97 x 69.92,46.49,PaleTurquoise,Plastic,24 months,60 days,10.86,null,2025-10-02,false
af734fb5-076e-4454-9379-13d0b4376cb9,8f7ede44-518d-410f-94df-200300601f83,8bcd2ff8-5c09-41e4-86c0-4cbff06a3906,2025-10-02T05:35:39.062Z,4.0,497.01,1988.0400390625,Credit Card,"2604 Peterson Branch Justinburgh, UT 23678",199,yes,Yourself management may choice particularly order.,harolddavis,royrobin@example.org,104ff6d71c7fbd29a5aa69b4b672fd83845649afce7b808276a2437906d06bda,Pamela Wilson,1938-09-14,Male,416-918-2710,"2604 Peterson Branch Justinburgh, UT 23678",South Lauraberg,New Mexico,Suriname,25861,2020-07-07,2025-08-21,Inactive,Admin,Bank Transfer,3780.42,true,List(),"List(7a92f37d-267e-4a83-85d9-47181cd38e8e, b478241d-7168-4cec-a8f3-75caf29e2a27, a274a939-b2b9-435f-aad7-f2cb81f64f91, 6a68ac6f-4246-4f5b-8f28-a990f8e5a5fa, 78b2d1b8-7ad4-49aa-ba74-660e6bb3a6fe)",Building Blocks,Toys,Puzzles,BrandVV,Safe and durable materials.,497.01,0.0,2,eMd-45375023,https://placekitten.com/774/668,3.200000047683716,4306,44ca6767-7f28-4602-bdbe-cb0796ef6a16,2024-09-24,56.74 x 25.08 x 70.06,1.45,Gold,Plastic,6 months,No returns,26.95,"List(keep, he)",2025-10-02,false
0af011c7-f021-4689-99ed-e6d105d88910,1cdcdc93-5d2e-4448-9ce0-f5df2840d3cd,371f1866-6c63-42c2-8dfb-2c18f1984a4c,2025-10-02T19:22:48.037Z,

### 9.3 Crear el Monitoring

Un Lakehouse Monitor analiza periódicamente una tabla (normalmente una Gold o Silver) y genera métricas como:

- Distribuciones de columnas (mean, std, min, max, nulls, etc.)
- Detección de data drift (comparar cómo cambian los datos entre días/horas)
- Detección de anomalías en series temporales
- Conteo de outliers, duplicados, tipos erróneos
- Freshness (tiempo desde la última actualización)

Databricks crea internamente:
- Un objeto “Lakehouse Monitor” vinculado a esa tabla.
- Dos tablas de resultados:
    - ``_profile_metrics`` → estadísticas por columna.
    - ``_drift_metrics`` → comparación temporal entre periodos.
- Un schedule para refrescar esas métricas.
- Un dashboard autogenerado (visible desde Catalog → Quality).

Para crear un Monitoring, podemos elegir entre tres tipos de perfiles diferentes:  
1. **Timeseries**: agrega métricas de calidad a lo largo de ventanas de tiempo.  
2. **Snapshot**: calcula métricas de calidad sobre toda la tabla.  
3. **Inference**: realiza un seguimiento del desplazamiento del modelo (*model drift*) y su rendimiento a lo largo del tiempo.  

Dado que estamos monitorizando datos de transacciones y la tabla contiene una columna de marca de tiempo, el tipo *Timeseries* es el más adecuado para este caso.  
Para otros tipos de análisis, consulta la documentación de Lakehouse Monitoring ([AWS](https://docs.databricks.com/en/lakehouse-monitoring/create-monitor-ui.html#profiling) | [Azure](https://learn.microsoft.com/en-us/azure/databricks/lakehouse-monitoring/create-monitor-ui#profiling)).

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.catalog import MonitorTimeSeries
import os

In [0]:
# Define ventanas de tiempo para agregar métricas durante
GRANULARITIES = ["1 day"]                       

# Opcionalmente, define expresiones para segmentar los datos con
SLICING_EXPRS = ["Category='Toys'"]  

In [0]:
# Debe tener privilegios `USE CATALOG` sobre el catálogo y debe tener privilegios `USE SCHEMA` sobre el esquema.
TABLE_NAME = f"{catalog}.{dbName}.silver_transaction"

# Define la columna de time_Stamp
TIMESTAMP_COL = "TransactionDate"

# Habilitar la fuente de datos de cambios (CDF) para procesar incrementalmente los cambios en la tabla y lograr una ejecución más eficiente
display(spark.sql(f"ALTER TABLE {TABLE_NAME} SET TBLPROPERTIES (delta.enableChangeDataFeed = true)"))

In [0]:
# Crea un monitor utilizando un perfil de tipo Serie temporal. Una vez completada la actualización inicial, podrás ver el panel generado automáticamente en la pestaña Calidad de la tabla en el Explorador de catálogos.
print(f"Creating monitor for {TABLE_NAME}")

w = WorkspaceClient()

try:
  lhm_monitor = w.quality_monitors.create(
    table_name=TABLE_NAME, # Always use 3-level namespace
    time_series = MonitorTimeSeries(
      timestamp_col=TIMESTAMP_COL,
      granularities=GRANULARITIES
    ),
    assets_dir = f"{os.getcwd()}/monitoring",
    output_schema_name=f"{catalog}.{dbName}"
  )
  
except Exception as lhm_exception:
  if "already exist" in str(lhm_exception).lower():
    print(f"Monitor for {TABLE_NAME} already exists, retrieving monitor info:")
    lhm_monitor = w.quality_monitors.get(table_name=f"{TABLE_NAME}")

  else:
    raise lhm_exception

El siguiente paso espera a que se cree el monitor y, a continuación, a que finalice el cálculo inicial de las métricas.

Nota: La creación y actualización del monitor puede tardar más de 10 minutos.

In [0]:
import time
from databricks.sdk.service.catalog import MonitorInfoStatus, MonitorRefreshInfoState

In [0]:
# Wait for monitor to be created
lhm_monitor = w.quality_monitors.get(table_name=f"{TABLE_NAME}")
while lhm_monitor.status == MonitorInfoStatus.MONITOR_STATUS_PENDING:
  lhm_monitor = w.quality_monitors.get(table_name=f"{TABLE_NAME}")
  time.sleep(10)

assert lhm_monitor.status == MonitorInfoStatus.MONITOR_STATUS_ACTIVE, "Error creating monitor"

refreshes = w.quality_monitors.list_refreshes(table_name=f"{TABLE_NAME}").refreshes
assert(len(refreshes) > 0)

run_info = refreshes[0]
while run_info.state in (MonitorRefreshInfoState.PENDING, MonitorRefreshInfoState.RUNNING):
  run_info = w.quality_monitors.get_refresh(table_name=f"{TABLE_NAME}", refresh_id=run_info.refresh_id)
  time.sleep(30)

assert run_info.state == MonitorRefreshInfoState.SUCCESS, "Monitor refresh failed"

### 9.4 Orientación sobre la tabla de métricas de perfil


Esta tabla contiene las estadísticas que genera el monitor de calidad sobre tu tabla (por ejemplo: medias, nulos, desviaciones, etc.).  
Cada fila representa una combinación de **columna + periodo de tiempo + grupo (slice)**. 



---

#### Columnas principales

| Columna | Descripción |
|----------|-------------|
| **log_type** | Indica de dónde vienen los datos:<br>- `INPUT`: estadísticas de la tabla actual.<br>- `BASELINE`: estadísticas de referencia (la tabla con la que se compara). |
| **column_name** | Nombre de la columna analizada. Si el valor es `:table`, la métrica se calcula sobre toda la tabla (por ejemplo, número total de filas). |
| **granularity** | Solo aparece en monitores de tipo *TimeSeries*. Indica la granularidad temporal (por ejemplo, `1 day`). En las filas de `BASELINE` aparece como `null`. |
| **window** | Muestra el periodo temporal específico (por ejemplo, `2025-10-31`). En las filas de `BASELINE` aparece como `null`. |
| **slice_key / slice_value** | Si el monitor divide los datos en grupos (por ejemplo, por país o región), estas columnas indican el nombre del grupo y su valor. Si no hay agrupación, ambos aparecen como `null`. |

---

#### Interpretación

Cada fila indica algo del estilo:

> Para la columna X, en el periodo Y y (si aplica) en el grupo Z, las estadísticas calculadas fueron las siguientes (media, nulos, etc.).

Las filas con:
- `log_type = BASELINE` → métricas de referencia.  
- `column_name = :table` → métricas globales (no asociadas a una columna concreta).


In [0]:
# Display profile metrics table
profile_table = lhm_monitor.profile_metrics_table_name  
display(spark.sql(f"SELECT * FROM {profile_table}"))

### 9.5 Orientacion sobre la tabla `_drift_metrics`

Esta tabla contiene las métricas que muestran **cómo cambian los datos con el tiempo** o **respecto a una tabla de referencia (baseline)**.  
Cada fila representa una combinación de **columna + periodo de tiempo + grupo (slice)**.

---

#### Columnas principales

| Columna | Descripción |
|----------|-------------|
| **drift_type** | Indica con qué se compara la tabla actual:<br>- `CONSECUTIVE`: comparación con la ventana temporal anterior.<br>- `BASELINE`: comparación con la tabla de referencia. |
| **column_name** | Nombre de la columna analizada. Si el valor es `:table`, la métrica se calcula sobre toda la tabla (por ejemplo, número total de filas o tasa de cambio global). |
| **granularity** | Solo aparece en monitores de tipo *TimeSeries*. Indica la granularidad temporal (por ejemplo, `1 day`). |
| **window** | Ventana temporal de los datos actuales (por ejemplo, `2025-10-31`). |
| **window_cmp** | Ventana con la que se hace la comparación. Si la comparación es con la tabla de referencia, este valor aparece como `null`. |
| **slice_key / slice_value** | Si el monitor agrupa los datos (por ejemplo, por país o categoría), estas columnas indican el grupo y su valor. Si no hay agrupación, ambos aparecen como `null`. |

---

#### Interpretación

Cada fila indica algo del estilo:

> Para la columna X, en la ventana Y, comparada con la ventana Z (o con la baseline), las métricas de deriva calculadas fueron las siguientes.

Las filas con:
- `drift_type = CONSECUTIVE` → comparan una ventana con la anterior.  
- `drift_type = BASELINE` → comparan con la tabla de referencia.  
- `column_name = :table` → métricas globales, no asociadas a una columna concreta.


In [0]:
# Display the drift metrics table
drift_table = lhm_monitor.drift_metrics_table_name  
display(spark.sql(f"SELECT * FROM {drift_table}"))

Veamos las métricas de perfil calculadas para la columna «TotalPurchaseAmount». Tenga en cuenta que las métricas se calculan para toda la tabla y para cada columna individual. Analizaremos esto con más detalle al agregar métricas personalizadas.

In [0]:
display(spark.sql(f"SELECT * FROM {profile_table} where column_name = 'TotalPurchaseAmount'"))

### 9.6 Visualizar el panel autogenerado

Cuando finaliza el primer *refresh*, puedes ver el **dashboard autogenerado** desde la pestaña **Quality** de la tabla `silver_transactions` en el Catalog Explorer.  
Este panel muestra distintas secciones con las métricas del monitor:

1. **Volumen de datos**: Permite comprobar si el volumen de transacciones es el esperado o si ha habido variaciones por estacionalidad.  
2. **Integridad de datos**: Muestra las columnas con un alto porcentaje de valores nulos o ceros y su evolución a lo largo del tiempo.  
3. **Cambio en distribución numérica**: Detecta anomalías en variables numéricas y visualiza su rango de valores a lo largo del tiempo.  
4. **Cambio en distribución categórica**: Identifica anomalías en variables categóricas (por ejemplo, `PreferredPaymentMethod`) y muestra cómo cambia la distribución de valores con el tiempo.  
5. **Perfilado**: Permite explorar el perfil de datos numéricos y categóricos a lo largo del tiempo.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/lhm/lhm_dashboard-1.png?raw=true" width="800px"/>


### 9.7 Investigación del Dashboard

Sin necesidad de herramientas adicionales ni configuraciones complejas, **Lakehouse Monitoring** permite perfilar, diagnosticar y controlar la calidad de los datos directamente dentro de la plataforma **Databricks Data Intelligence Platform**.

A partir del dashboard generado, podemos responder las tres preguntas iniciales:

1. **¿Qué porcentaje de valores nulos se ha introducido en esta columna? ¿Es algo normal?**  
   > En la sección *% Nulls* se observa que `PreferredPaymentMethod` pasó de un 10 % a alrededor de un 40 %.

2. **Si no es normal, ¿cuál fue la causa raíz de este problema de integridad?**  
   > En la sección *Categorical Distribution Change* vemos que tanto `PreferredPaymentMethod` como `PaymentMethod` muestran una alta deriva en la última ventana temporal.  
   > El *heatmap* revela que **Apple Pay** se añadió recientemente como nuevo valor de `PaymentMethod` al mismo tiempo que comenzaron a aparecer valores `null` en `PreferredPaymentMethod`.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/lhm/lhm_dashboard-2.png?raw=true" width="800px" style="float:right"/>

3. **¿Qué activos posteriores podrían haberse visto afectados por este problema?**  
   > Como Lakehouse Monitoring se integra con **Unity Catalog**, puedes usar el **Lineage Graph** para identificar las tablas descendientes que han sido impactadas.

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/lhm/lhm_lineage.png?raw=true" width="800px" style="float:right"/>

Tal como se muestra en este ejemplo, puedes **detectar proactivamente los problemas de calidad antes de que afecten a los procesos posteriores**.  
Comienza a usar **Lakehouse Monitoring (General Availability)** en tu entorno

### 9.8 Quality Expectations

#### 9.8.1 ¿Qué son las *Data Quality Expectations*?

Las **Data Quality Expectations** son reglas de validación que puedes definir directamente sobre una tabla Delta dentro de **Unity Catalog**.
Sirven para **controlar automáticamente la calidad de los datos** y mostrar visualmente si una tabla cumple o no los criterios definidos (✅ o ❌).

---



%md
<img src="https://raw.githubusercontent.com/jmartinezceste/251101Course_UC_Ceste/main/photos/quality_expectativas.png" width="800">

#### 9.8.2 Concepto

Una *expectation* es una condición SQL que debe cumplirse.
Databricks la evalúa cada vez que los datos se escriben o actualizan.

```sql
ALTER TABLE main.sales.transactions
ADD EXPECTATION no_null_customer CHECK (customer_id IS NOT NULL);
```

Si la condición se cumple → la tabla se marca como **OK (✅)**
Si falla → se marca como **Failed (❌)** y el evento queda registrado.

---

#### ⚙️ Cómo funcionan

* **Definición:** Se añaden a nivel de tabla (una o varias por tabla).
* **Evaluación:** Se ejecutan automáticamente en las operaciones *write* o de forma programada.
* **Acción:** Puedes decidir si solo se registran los fallos (*LOG ONLY*) o si bloquean la carga (*FAIL ON WRITE*).
* **Visualización:** En *Catalog Explorer → Quality*, o directamente en la vista del schema (tick o X).

---

#### 🧠 Ejemplos típicos de uso

| Tipo de validación      | Ejemplo de regla                                | Descripción                                 |
| ----------------------- | ----------------------------------------------- | ------------------------------------------- |
| **Completeness**        | `COUNT_IF(ProductID IS NULL) / COUNT(*) < 0.05` | Menos del 5 % de valores nulos.             |
| **Rango de valores**    | `price BETWEEN 0 AND 10000`                     | El precio está en un rango válido.          |
| **Freshness**           | `event_date >= current_date() - INTERVAL 1 DAY` | Los datos son recientes.                    |
| **Consistencia lógica** | `order_date <= delivery_date`                   | La fecha de entrega es posterior al pedido. |

---

#### 📊 Resultados

Los resultados de cada evaluación se guardan en tablas del sistema:

```sql
SELECT * 
FROM system.data_quality_monitoring.expectation_results
WHERE table_name = 'silver_transaction';
```

---

#### 🎯 Beneficios

* Control visual y automático de calidad (✅/❌).
* Registro histórico de violaciones.
* Integración con *Data Quality Monitoring* y *Governance Hub*.
* Menos validaciones manuales o scripts externos.


#### 9.8.2 Pasos en la interfaz visual (Catalog Explorer)

---

1. **Abrir la tabla**

   * Ve a **Catalog** → selecciona tu catálogo `main_monitoring`.
   * Entra en el **schema** `demo_monitoring`.
   * Haz clic sobre la tabla `silver_transaction`.

2. **Acceder a la pestaña de calidad**

   * En la parte superior verás pestañas como *Overview · Lineage · Permissions · Quality*.
   * Haz clic en **Quality**.

3. **Añadir una Expectation**

   * Pulsa **Add Expectation**.
   * En **Name** escribe:
     `valid_productid_completeness`
   * En **Condition**, pega esta expresión:

     ```sql
     (COUNT_IF(ProductID IS NULL) / COUNT(*)) < 0.05
     ```
   * En **Action on violation**, selecciona:

     * **Log only** → solo registra fallos (verás ❌ pero no se bloquea la escritura).
     * **Fail on write** → bloquea la escritura si no se cumple.

4. **Guardar la Expectation**

   * Haz clic en **Save**.
   * Databricks empezará a monitorizar la tabla en cada actualización.

5. **Verificar los resultados**

   * En la pestaña **Quality** podrás ver si la expectation pasa o falla.
   * En la vista general del *schema*, la tabla mostrará:

     * ✅ si cumple.
     * ❌ si falla.

---

### 🧠 Tips

* Si no ves **Add Expectation**:

  * Verifica que tienes permisos **MANAGE TABLE**.
  * Asegúrate de que el *schema* tiene activado **Data Quality Monitoring**
    (*Schema → Details → Enable Data Quality Monitoring*).
* Los resultados también se guardan en `system.data_quality_monitoring.expectation_results`.

---
